In [3]:
import matplotlib
import operator
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib

# Set 
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['font.family'] = "serif"
matplotlib.rcParams['font.size'] = 10

# import custom functions
from functions_statistical_performance import *

In [4]:
# True to exactly reproduce the results presented in the paper (same classifiers)
reproduceflag=False

In [5]:
# function to predict over a generic db when you already have the classifiers
# it provides the average FPro 
def ClassifyDB(db, model_per_fold,nut_sel):
    dbsel=db.loc[:,nut_sel]
    Xnut=dbsel.values    
    
    indfold=0
    for model in model_per_fold:
        indfold+=1
        y_pred = model.predict(Xnut)
        y_probs= model.predict_proba(Xnut)
        db['classf'+str(indfold)]=y_pred
        db['p1f'+str(indfold)]=y_probs[:,0]
        db['p2f'+str(indfold)]=y_probs[:,1]
        db['p3f'+str(indfold)]=y_probs[:,2]
        db['p4f'+str(indfold)]=y_probs[:,3]
        db['FProf'+str(indfold)]=(1-db['p1f'+str(indfold)]+db['p4f'+str(indfold)])/2
     
    for p in range(1,5):
        db['p'+str(p)]=db.loc[:, ['p'+str(p)+'f1', 'p'+str(p)+'f2', 'p'+str(p)+'f3','p'+str(p)+'f4', 'p'+str(p)+'f5' ]].mean(axis=1)
        db['std_p'+str(p)]=db.loc[:, ['p'+str(p)+'f1', 'p'+str(p)+'f2', 'p'+str(p)+'f3','p'+str(p)+'f4', 'p'+str(p)+'f5' ]].std(axis=1)


    db['FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5' ]].mean(axis=1)
    db['std_FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].std(axis=1)
    db['min_FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].min(axis=1)


    db['max_p']=db.loc[:, ['p1', 'p2', 'p3','p4' ]].idxmax(axis=1)
    db['class']=[int(s[1])-1 for s in db.loc[:, ['p1', 'p2', 'p3','p4' ]].idxmax(axis=1)]
    db['min_in_which_fold']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].idxmin(axis=1)
    db['min_fold_id']=[int(s[-1]) for s in db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].idxmin(axis=1)]
    db['min_class']=[db['classf'+str(db['min_fold_id'].iloc[n])].iloc[n] for n in range(db.shape[0])]
    
    for ind in range(1,5):
        db['p'+str(ind)+'_minFPro']=[db['p'+str(ind)+'f'+str(db['min_fold_id'].iloc[n])].iloc[n] for n in range(db.shape[0])]
    

    return db

# Load 2009-2010 Data

In [6]:
# log-transformed nutrient content per 100 grams
RFFNDDS=pd.read_csv('./input_data/FNDDS_2009-10_Training_Data.csv')
RFFNDDS

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,Total flavones,Total flavonols,Total isoflavones,Food_code,Main_food_description,catnumb,catname,novaclass,macroclass,pythonlabel
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,-20.0,-20.0,-20.0,2047,"Salt, table",0,addition1516,2,Other,1
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,-20.0,-20.0,-20.0,11000000,"Milk, human",9602,Human milk,1,Milk,0
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,-20.0,-20.0,-20.0,11100000,"Milk, NFS",1004,"Milk, reduced fat",1,Milk,0
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,-20.0,-20.0,-20.0,11111000,"Milk, cow's, fluid, whole",1002,"Milk, whole",1,Milk,0
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,-20.0,-20.0,-20.0,11111100,"Milk, cow's, fluid, whole, low-sodium",1002,"Milk, whole",1,Milk,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,-20.0,-20.0,-20.0,94100200,"Water, bottled, sweetened, with low or no calo...",7802,Flavored or carbonated water,4,Other,3
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,-20.0,-20.0,-20.0,94100300,"Water, fruit flavored, sweetened, with high fr...",7802,Flavored or carbonated water,4,Other,3
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,-20.0,-20.0,-20.0,94210100,Propel Water,7804,Enhanced or fortified water,4,Other,3
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,-20.0,-20.0,-20.0,94210200,Glaceau Water,7804,Enhanced or fortified water,4,Other,3


In [7]:
# total nutrient list
nl=list(RFFNDDS.columns)
nl=nl[0:-7]
len(nl)

99

# Whole Nutrient Panel Analysis

In [8]:
if reproduceflag:
    model_per_foldWNP=joblib.load("./paper_classifiers_outcomes/FNDDS0910_99Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDS[RFFNDDS.pythonlabel!=-1]
    data2train['pythonlabel'] = np.random.permutation(data2train['pythonlabel'].values)
    X=data2train.loc[:, 'Protein': 'Total isoflavones']
    y=data2train['pythonlabel']
    smoteflag=True
    (performancesAUCWNP, performancesAUPWNP, splitsWNP, model_per_foldWNP)=AUCAUPkfold(X,y,smoteflag)


C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


AUC: average over the folds
[0.96868092 0.99828739 0.95209191 0.91058521]
AUC: std over the folds
[0.00192586 0.00036525 0.00497538 0.00848706]
AUP: average over the folds
[0.89794191 0.99418666 0.85456313 0.74111451]
AUP: std over the folds
[0.0140237  0.00116297 0.01272721 0.00812703]
Balanced Accuracy: average over the folds
0.8623357198077375
Balanced Accuracy: std over the folds
0.006153650371845238


In [9]:
# summary database
df=RFFNDDS.copy()
df=ClassifyDB(df,model_per_foldWNP, nl)
df.to_csv('./output_data/FNDDS_2009-10_99_Nutrients_ensemble_5folds_shuffled.csv')
df

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,0.599917,p4,3,FProf4,4,3,0.253250,0.110000,0.183667,0.453083
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,0.578646,p4,3,FProf5,5,3,0.257917,0.066875,0.260000,0.415208
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,0.035625,p1,0,FProf1,1,0,0.945000,0.005000,0.033750,0.016250
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,0.505000,p3,2,FProf2,2,2,0.015000,0.000000,0.960000,0.025000
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,0.587917,p4,3,FProf4,4,2,0.082500,0.015000,0.644167,0.258333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,0.633181,p4,3,FProf2,2,3,0.137542,0.105000,0.353554,0.403904
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,0.492500,p3,2,FProf5,5,2,0.020000,0.005000,0.970000,0.005000
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,0.505000,p3,2,FProf2,2,2,0.050000,0.035000,0.855000,0.060000
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,0.495000,p3,2,FProf2,2,2,0.015000,0.000000,0.980000,0.005000


# Standard Release FNDDS (for releases without flavonoids)

In [10]:
RFFNDDSSR=RFFNDDS.loc[:, 'Protein': 'Fatty acids, total polyunsaturated']
StandardNutPanel=list(RFFNDDSSR.columns)
RFFNDDSSR['pythonlabel']=RFFNDDS['pythonlabel'].copy()
print('Number of Nutrients in the Standard Nutrient Panel')
print(len(StandardNutPanel))

Number of Nutrients in the Standard Nutrient Panel
62


In [11]:
if reproduceflag:
    model_per_fold62nut=joblib.load("./paper_classifiers_outcomes/FNDDS0910_62Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDSSR[RFFNDDSSR.pythonlabel!=-1]
    data2train['pythonlabel'] = np.random.permutation(data2train['pythonlabel'].values)
    X=data2train.loc[:, 'Protein': 'Fatty acids, total polyunsaturated']
    y=data2train['pythonlabel']
    smoteflag=True
    (performancesAUC62nut, performancesAUP62nut, splits62nut, model_per_fold62nut)=AUCAUPkfold(X,y,smoteflag)

C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


AUC: average over the folds
[0.96363004 0.9985819  0.95021388 0.91534943]
AUC: std over the folds
[0.00233709 0.00051599 0.00380143 0.00328261]
AUP: average over the folds
[0.88410652 0.99450824 0.86225165 0.75660934]
AUP: std over the folds
[0.01256344 0.00265063 0.01652697 0.01461579]
Balanced Accuracy: average over the folds
0.8644664605111313
Balanced Accuracy: std over the folds
0.0045025375454659095


In [12]:
# summary database
df=RFFNDDSSR.copy()
df=ClassifyDB(df,model_per_fold62nut,StandardNutPanel)
df.to_csv('./output_data/FNDDS_2009-10_62Nutrients_ensemble_5folds_shuffled.csv')
df

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,0.624851,p4,3,FProf3,3,3,0.223333,0.100,0.203631,0.473036
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,0.616333,p4,3,FProf4,4,3,0.237000,0.055,0.238333,0.469667
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,0.528750,p3,2,FProf2,2,2,0.043750,0.000,0.855000,0.101250
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,0.870923,p4,3,FProf4,4,3,0.066786,0.000,0.124583,0.808631
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,0.586542,p4,3,FProf4,4,3,0.265000,0.005,0.291917,0.438083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,0.733167,p4,3,FProf2,2,3,0.133333,0.000,0.267000,0.599667
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,0.615000,p3,2,FProf5,5,2,0.015000,0.015,0.725000,0.245000
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,0.637708,p4,3,FProf2,2,3,0.177500,0.000,0.369583,0.452917
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,0.714167,p4,3,FProf3,3,3,0.085000,0.030,0.371667,0.513333


# Classify FNDDS 2015-2016 with 62 Nutrients

In [13]:
# log-transformed nutrient content per 100 grams
RFFNDDS20152016=pd.read_csv('./input_data/FNDDS_2015-16.csv')
df=ClassifyDB(RFFNDDS20152016,model_per_fold62nut,StandardNutPanel)
df.to_csv("./output_data/FNDDS_2015-16_62Nutrients_ensemble_5folds_shuffled.csv")
df

C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Food code,Main food description,year,cat_digit_1,cat_digit_2,cat_digit_3,cat_digit_4,cat_digit_5,WWEIA Category number,WWEIA Category description,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,11000000,"Milk, human",2015,Milk and Milk Products,Milks and milk drinks,"Milk, human",NaN,NaN,9602,Human milk,...,0.616333,p4,3,FProf4,4,3,0.237000,0.055,0.238333,0.469667
1,11100000,"Milk, NFS",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1004,"Milk, reduced fat",...,0.550625,p3,2,FProf5,5,2,0.070000,0.005,0.753750,0.171250
2,11111000,"Milk, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.870923,p4,3,FProf4,4,3,0.066786,0.000,0.124583,0.808631
3,11111100,"Milk, low sodium, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.586542,p4,3,FProf4,4,3,0.265000,0.005,0.291917,0.438083
4,11111150,"Milk, calcium fortified, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.870923,p4,3,FProf4,4,3,0.066786,0.000,0.124583,0.808631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8685,99991400,Cheese as ingredient in sandwiches,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.480000,p4,3,FProf4,4,0,0.465000,0.010,0.100000,0.425000
8686,99995000,Breading or batter as ingredient in food,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.598333,p4,3,FProf2,2,3,0.165000,0.265,0.208333,0.361667
8687,99995130,Wheat bread as ingredient in sandwiches,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.662500,p3,2,FProf5,5,2,0.030000,0.010,0.605000,0.355000
8688,99998130,Sauce as ingredient in hamburgers,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.483119,p4,3,FProf4,4,0,0.436881,0.005,0.155000,0.403119


# Nutrition Facts Analysis

In [14]:
nutrition_facts=['Protein','Total Fat','Carbohydrate','Sugars, total','Fiber, total dietary','Calcium','Iron', 'Sodium','Retinol','Carotene, beta',
 'Carotene, alpha','Cryptoxanthin, beta','Vitamin C','Cholesterol', 'Fatty acids, total saturated'];
VitaminA=['Retinol','Carotene, beta','Carotene, alpha','Cryptoxanthin, beta']
RFFNDDSNF=RFFNDDS[nutrition_facts]
RFFNDDSNF['Total Vitamin A']=RFFNDDS[VitaminA].apply(np.exp).sum(axis=1).apply(np.log)
RFFNDDSNF=RFFNDDSNF.drop(VitaminA, axis=1)
nutrition_facts=list(RFFNDDSNF.columns)
print('Number of Nutrients in the Nutrition Facts Panel')
print(len(nutrition_facts))
RFFNDDSNF['pythonlabel']=RFFNDDS['pythonlabel'].copy()

Number of Nutrients in the Nutrition Facts Panel
12


C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
if reproduceflag:
    model_per_fold12nut=joblib.load("./paper_classifiers_outcomes/FNDDS0910_12Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDSNF[RFFNDDSNF.pythonlabel!=-1]
    data2train['pythonlabel'] = np.random.permutation(data2train['pythonlabel'].values)
    X=data2train.loc[:, 'Protein': 'Total Vitamin A']
    y=data2train['pythonlabel']
    smoteflag=True
    (performancesAUC12nut, performancesAUP12nut, splits12nut, model_per_fold12nut)=AUCAUPkfold(X,y,smoteflag)

In [ ]:
# summary database
df=RFFNDDSNF.copy()
df=ClassifyDB(df,model_per_fold12nut,nutrition_facts)
df.to_csv('./output_data/FNDDS_2009-10_12Nutrients_ensemble_5folds_shuffle.csv')
df